In [ ]:
from pathlib import Path

import polars as pl

from timma import crosstab_income_residency

In [ ]:
dir = r"X:\Projects\TIMMA\Round7\Round7_2040_weekday_ubi_currentandlowincomeresidents"
trips_joined_filename = "TRIPMC1-simplified.parquet"
trips_joined_filepath = Path(dir, "summaries", trips_joined_filename)

In [ ]:
trips = pl.read_parquet(trips_joined_filepath).with_columns(
    # I don't want to deal with trying to order a Categorical right now
    pl.col("income_quintile").cast(pl.Utf8).cast(pl.Int8),
    # casting to str from Categorical to allow is_in()
    pl.col("trip_mode").cast(pl.Utf8)
)
trips

In [ ]:
from_ti_filter = (pl.col("origin") == "TI")
to_ti_filter = (pl.col("destination") == "TI")
ti_trips_inc_intra_ti_filter = from_ti_filter | to_ti_filter
ti_trips_exc_intra_ti_filter = from_ti_filter ^ to_ti_filter  # XOR

automobile_filter = pl.col("trip_mode").is_in(["auto-DA", "auto-SR2", "auto-SR3"])
transit_filter = (pl.col("trip_mode") == "transit")

daytime_filter = (1 < pl.col("mOdt")) & (pl.col("mOdt") < 5)

In [ ]:
crosstab_income_residency(trips.filter(to_ti_filter))

In [ ]:
crosstab_income_residency(trips.filter(from_ti_filter))

In [ ]:
crosstab_income_residency(
    trips.filter(~from_ti_filter & to_ti_filter & ~daytime_filter))

In [ ]:
crosstab_income_residency(
    trips.filter(from_ti_filter & ~to_ti_filter & ~daytime_filter))

In [ ]:
crosstab_income_residency(
    trips.filter(ti_trips_exc_intra_ti_filter & ~daytime_filter))

In [ ]:
crosstab_income_residency(
    trips.filter(ti_trips_inc_intra_ti_filter & ~daytime_filter))

In [ ]:
(crosstab_income_residency(trips.filter(
    ti_trips_exc_intra_ti_filter & daytime_filter &
    (pl.col("trip_mode") == "auto-DA")
)) + crosstab_income_residency(trips.filter(
    ti_trips_exc_intra_ti_filter &  daytime_filter &
    (pl.col("trip_mode") == "auto-SR2")
)) / 2 + crosstab_income_residency(trips.filter(
    ti_trips_exc_intra_ti_filter &  daytime_filter &
    (pl.col("trip_mode") == "auto-SR3")
)) / 3.5)